# Lab notebook: intro lab 6 

Written by Dana Griffith

5BL spring 2022

Last updated: 02/10/2022 at 2pm 

In [3]:
import numpy as np
import math

In [12]:
#DATA 
#Each trial is the time it takes for 10 oscillations
#3 trials per mass 

mass = [100, 200] #grams
mass100 = [4.78,4.81,4.72,4.81,4.78,4.79,4.72] #seconds, 3 trials for 100g mass
mass200 = [6.44,6.5,6.47,6.38,6.34,6.47,6.5] #seconds, 3 trials for 200g mass

In [17]:
#CALCULATIONS
#Average, st dev, st error, and total error for each mass

#100g mass trials (n=1)
avg100 = np.mean(mass100) #average
stdev100 = np.std(mass100) #standard dev
sterr100 = stdev100/np.sqrt(7) #standard error = std dev / sqrt(N trials)
toterr100 = sterr100**2 #total error = (standard error)^2

#200g mass trials (n=2)
avg200 = np.mean(mass200) #average
stdev200 = np.std(mass200) #standard dev
sterr200 = stdev200/np.sqrt(7) #standard error = std dev / sqrt(N trials)
toterr200 = sterr200**2 #total error = (standard error)^2

In [18]:
#MOMENT OF INERTIA
I_holder = 100 #g*cm^2
dI_holder = 3.2

I_disk = 286.8 #g*cm^2
dI_disk = 6.6

In [34]:
#ERROR CALCULATIONS


#Period, error, fractional error, etc
#100g mass

#Period
period100 = avg100/10 #period = mean/10 oscillations
period100_2 = period100**2 #period^2

#Error in period
alphaT_100 = toterr100/10 #alphaT = total error/10 oscillations
alphaT2_100 = 2 * period100 * alphaT_100 #alpha_T2

#Moment of inertia (error, fractional error)
I100 = 1*(I_holder + I_disk) #times 1 because n=1
alpha_I100 = np.sqrt(dI_holder**2 + dI_disk**2) #I error
fractionalI100 = I100/alpha_I100 #fractional I error

#Ratio alpha_I/I to alpha_T^2/T^2:
ratio100 = fractionalI100/(alphaT2_100/period100_2) #ratio of fractional I error to fractional period^2 error




#200g mass

#Period
period200 = avg200/10 #period = mean/10 oscillations
period200_2 = period200**2 #period^2

#Error in period
alphaT_200 = toterr200/10 #alphaT = total error/10 oscillations
alphaT2_200 = 2 * period200 * alphaT_200 #alpha_T2

#Moment of inertia (error, fractional error)
I200 = 2*(I_holder + I_disk) #times 2 because n=2
alpha_I200 = np.sqrt(dI_holder**2 + (2*dI_disk)**2)
fractionalI200 = I200/alpha_I200

#Ratio alpha_I/I to alpha_T^2/T^2:
ratio200 = fractionalI200/(alphaT2_200/period200_2)

# Chi squared zero

In [31]:
#LINEAR REGRESSION
#use polyfit of order 1 (aka a linear fit)
#polyfit info: https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html

linest = np.polyfit([I100,I200],[period100_2,period100_2],1,full=True)
print('Polynomial coefficients:',linest[0])
print('Residuals:',linest[3])

Polynomial coefficients: [ -3.20906690e-20   2.27801653e-01]
Residuals: [ 1.39595247  0.2265319 ]


In [35]:
#CHI SQUARED ZERO
#We will now calculate the 

#n=1 (100g mass)
y_x100 = linest[0][0]*period100_2 + linest[3][0] #y(x)
res100 = linest[3][0] #residual 
normres100 = res100/alpha_I100 #normalized residual 

#will need these error values later
alphaI0_100 = alphaT2_100 * linest[0][0] #alpha_I0
alphatau100 = alpha_I100 * alphaI0_100 #alpha_tau


#n=2 (200g mass)
y_x200 = linest[0][0]*period200_2 + linest[3][0]
res200 = linest[3][1]
normres200 = res200/alpha_I200

#will need these error values later
alphaI0_200 = alphaT2_200 * linest[0][0]
alphatau200 = alpha_I200 * alphaI0_200

#reduced chi squared
redchisq = np.sum([normres100,normres200])/1 #div by 1 from: n=2 masses - 1 

In [36]:
#Plot of torsional oscillator
#y axis: moment of inertia
#x axis: period^2 

# Chi squared weighted

In [44]:
#n=1
w100 = 1/(alphatau100**2) #weight
xyw100 = period100_2 * I100 * w100 #xyw
x2w100 = period100_2**2 * w100 #x^2 * w
whalf100 = 1/alphatau100 #w^(0.5)

#n=2
w200 = 1/(alphatau200**2) #weight
xyw200 = period200_2 * I200 * w200
x2w200 = period200_2**2 * w200
whalf200 = 1/alphatau200



#will need these sums to calculate the slope
xyw = np.sum([xyw100,xyw200]) #etc
w = np.sum([w100, w200]) 
x2w = np.sum([x2w100,x2w200])
zeroslope = xyw/x2w #weighted LINEAR, zero intercept



#n=1
ressq100 = I100 - (zeroslope*period100_2)**2 #residual squared
xsq100 = period100_2**2 #x^2
xw100 = period100_2 * w100 #xw
wy100 = I100 * w100 #wy

#n=2
ressq200 = I200 - (zeroslope*period200_2)**2
xsq200 = period200_2**2 #x^2
xw200 = period200_2 * w200
wy200 = I200 * w200


#calculate more parts of the chi squared eqn
xw = np.sum([xw100, xw200])
wy = np.sum([wy100,wy200])

2065.10936858


In [47]:
#Weighted slope

slope = ((w*xyw)-(xw*wy))/((w*x2w)-(xw*xw)) #WEIGHTED nonzero slope
slope_err = np.sqrt(w/((w*x2w)-(xw*xw))) #error in slope
intercept = ((x2w*wy)-(xw*xyw))/((w*x2w)-(xw*xw))
intercept_err = np.sqrt(x2w/((w*x2w)-(xw*xw))) #error in intercept

#n=1
y_xwtd100 = slope*period100_2 + intercept #y(x) weighted
reswtd100 = I100 - y_xwtd100 #weighted residual
normreswtd100 = reswtd100/alphatau100 #normalized weighted residual
normressqwtd100 = normreswtd100**2 #normalized residual squared

#n=2
y_xwtd200 = slope*period200_2 + intercept
reswtd200 = I100 - y_xwtd200
normreswtd200 = reswtd200/alphatau200
normressqwtd200 = normreswtd200**2

chisqwtd = np.mean([normressqwtd100,normressqwtd200]) #weighted chi^2